## RNN in tensorflow

In [1]:
import tensorflow as tf
from  tensorflow.contrib import rnn
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# 训练参数
learning_rate = 0.001
training_steps = 1000
batch_size = 128
display_step = 200

In [4]:
# 神经网络参数
num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

In [5]:
X = tf.placeholder(tf.float32, [None ,timesteps, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [6]:
# 定义权重
weights = {
    'out':tf.Variable(tf.random_normal([num_hidden, num_classes]))
}

biases = {
    'out':tf.Variable(tf.random_normal([num_classes]))
}

In [7]:
def RNN(x, weights, biases):
    x = tf.unstack(x, timesteps, 1)
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [8]:
# 输出
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

In [9]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss_op)

#评估模型
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
init = tf.global_variables_initializer()

In [11]:
# 开始训练
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(training_steps):
        batch_x, batch_y = mnist.train.next_batch(batch_size)        
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(training_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 0, Minibatch Loss= 3.3789, Training Accuracy= 0.148
Step 200, Minibatch Loss= 2.1731, Training Accuracy= 0.258
Step 400, Minibatch Loss= 1.9325, Training Accuracy= 0.359
Step 600, Minibatch Loss= 1.9758, Training Accuracy= 0.289
Step 800, Minibatch Loss= 1.7897, Training Accuracy= 0.406
Optimization Finished!
Testing Accuracy: 0.4609375
